## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [24]:
import numpy as np

A = np.random.randint(0, 1000, size=(1000000, ))
A

array([357, 995, 319, ..., 213, 125, 643])

In [25]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

%timeit f1(A)

448 ms ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai + 100
        acc += bi
    return acc/len(A)

%timeit f2(A)

403 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
def f3(A):
    return sum(A)/len(A)+100

%timeit f3(A)

124 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
def f4(A):
    return A.mean()+100

%timeit f4(A)

780 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%lprun -f f2 f2(A)

In [20]:
import numba

In [29]:
@numba.njit
def f5(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

%timeit f5(A)

438 µs ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [60]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)), columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)
df

,col1,col2,col3,col4,key
0,774,182,912,213,a
1,141,571,813,349,d
2,321,272,500,485,f
3,803,361,654,732,c
4,489,396,342,964,c
...,...,...,...,...,...
1999995,357,749,259,901,e
1999996,933,210,361,957,e
1999997,75,862,412,311,c
1999998,690,850,858,931,d


In [61]:
def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

g(df)

,col1,col2,col3,col4,key
0,774,182,912,213,a
6,141,519,45,664,a
36,560,377,965,897,a
37,740,318,285,773,a
48,870,362,121,860,a
...,...,...,...,...,...
1999986,620,373,452,169,e
1999991,811,71,902,392,e
1999992,803,841,795,552,e
1999995,357,749,259,901,e


In [62]:
%timeit g(df)

563 ms ± 7.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
def g1(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    return df[df['key'].isin(letters)]

g1(df)

,col1,col2,col3,col4,key
0,774,182,912,213,a
1,141,571,813,349,d
3,803,361,654,732,c
4,489,396,342,964,c
5,995,687,778,137,c
...,...,...,...,...,...
1999995,357,749,259,901,e
1999996,933,210,361,957,e
1999997,75,862,412,311,c
1999998,690,850,858,931,d


In [64]:
%timeit g1(df)

82.3 ms ± 873 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

In [1]:
!pip install line_profiler

In [2]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [10]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
